In [11]:
# funkcja generująca dane o konkretnej średniej i odchyleniu standardowym
# dzięki temu można wykorzystać wbudowane testy w R gdy nie mamy danych tylko ich statystyki opisowe
wygeneruj_dane <- function(srednia, odch_std, n) {
    if(n %% 2 == 1) {
        #1/(n-1) * (n-1) * a^2 = a^2 = odch_std^2 -> a = odch_std
        n_rep <- (n-1)/2
        dane <- c(0, rep(-odch_std, n_rep), rep(odch_std, n_rep))
    }else {
        #1 / (n-1) * n * a^2 = n/(n-1) * a^2 = odch_std^2 -> a = sqrt((n-1)/n) * odch_std
        n_rep <- n / 2
        rep_val <- sqrt((n-1) / n) * odch_std
        dane <- c(rep(rep_val, n_rep), rep(-rep_val, n_rep))
    }
    dane <- dane + srednia
}

* [Kowariancja](#Kowariancja)
* [Korelacja](#Korelacja)
    * [Test](#Test-na-korelację)
    * [Dla podanych danych](#Dla-podanych-danych)
    * [Dla podanych statystyk](#Dla-podanych-statystyk)
* [Testy dla dwóch populacji](#Testy-dla-dwóch-populacji)
    * [Jaki test wybrac](#Jaki-test-wybrac)
    * [Test Z sparowane](#Test-Z-sparowane)
    * [Test Z niesparowane](#Test-Z)
    * [Test T niesparowane](#Test-T)
        * [Dla rownych wariancji](#Dla-rownych-wariancji)
        * [Dla roznych wariancji](#Dla-roznych-wariancji)
* [Regresja liniowa](#Regresja-liniowa)
    * [Twierdzenia](#Twierdzenia)
        * Suma reszt = 0
        * Przechodzenie przez punkt (x_srednia, y_srednia)
        * Wyznaczanie współczynników B_0 i B_1 
    * [Reszta](#Reszta)
    * [SSE](#SSE)
    * [SSR](#SSR)
    * [SST](#SST)
    * [RMS](#RMS)
    * [Współczynnik determinacji](#Współczynnik-determinacji)

### Kowariancja <a class="anchor" id="Kowariancja"></a>
![](images/kowariancja.PNG)

In [12]:
x <- runif(10)
y <- runif(10)
print(paste("S_x,y =", cov(x,y)))

[1] "S_x,y = -0.00284820271124978"


### Korelacja
![](images/korelacja.png)

In [13]:
S_xy <- NULL
S_x <-  NULL
S_y <-  NULL
cor_xy <- S_xy / (S_x * S_y)
print(paste("r_x,y =", cor_xy))

[1] "r_x,y = "


In [14]:
x <- c(1,2,3,4)
y <- c(5,6,7,-8)
cor_xy <- cor(x, y, method = "pearson")
print(paste("r_x,y =", cor_xy))

[1] "r_x,y = -0.696106140174946"


![](images/korelacja_wlasciwosci.PNG)
![](images/przyklad_korelacja.PNG)
? - korelacja nie istnieje gdy jedna z zmiennych ma zerową wariancję (jest stała)
## Test na korelację
![](images/korelacja_test.PNG)
### Dla podanych statystyk

In [15]:
#cor_xy <- 
#n <- length(x)
n <- 10
poziom_istotnosci <- 0.01

cat(paste("Obliczona korelacja:", cor_xy, '\nPoziom istotnosci:', poziom_istotnosci, '\n\n'))

wart_stat_testowej <- cor_xy / (sqrt(1 - cor_xy ^ 2)) * sqrt(n-2)
cat(paste("T_(n-2) =", wart_stat_testowej, '\n'))

zbior_kryt_lewo <- qt(poziom_istotnosci, df = n - 2)
p_wart_lewo <- pt(wart_stat_testowej, df = n - 2)
cat(sprintf("
Test lewostronny
p-wartosc = %f
Zbior krytyczny: (-Inf, %f)
", p_wart_lewo, zbior_kryt_lewo))

zbior_kryt_prawo <- qt(1 - poziom_istotnosci, df = n - 2)
p_wart_prawo <- 1 - pt(wart_stat_testowej, df = n - 2)
cat(sprintf("
Test prawostronny
p-wartosc = %f
Zbior krytyczny: (%f, Inf)
", p_wart_prawo, zbior_kryt_prawo))


zbior_kryt_obu <- qt(poziom_istotnosci / 2, df = n - 2)
p_wart_obu <- 2 * (1 - pt(abs(wart_stat_testowej), df = n - 2))
cat(sprintf("
Test obustronny
p-wartosc = %f
Zbior krytyczny: (-Inf, %f) v (%f, Inf)
", p_wart_obu, zbior_kryt_obu, -zbior_kryt_obu))

Obliczona korelacja: -0.696106140174946 
Poziom istotnosci: 0.01 

T_(n-2) = -2.74241377865072 

Test lewostronny
p-wartosc = 0.012677
Zbior krytyczny: (-Inf, -2.896459)

Test prawostronny
p-wartosc = 0.987323
Zbior krytyczny: (2.896459, Inf)

Test obustronny
p-wartosc = 0.025354
Zbior krytyczny: (-Inf, -3.355387) v (3.355387, Inf)


### Dla podanych danych

In [16]:
x <- c(1,2,3,4)
y <- c(5,6,7,-8)
poziom_istotnosci <- 0.01


print("Test obustronny")
(cor.test(x, y, conf.level = 1-poziom_istotnosci))
print("Test lewostronny")
(cor.test(x, y, conf.level = 1-poziom_istotnosci, alternative = "less"))
print("Test prawostronny")
(cor.test(x, y, conf.level = 1-poziom_istotnosci, alternative = "greater"))

[1] "Test obustronny"



	Pearson's product-moment correlation

data:  x and y
t = -1.3712, df = 2, p-value = 0.3039
alternative hypothesis: true correlation is not equal to 0
99 percent confidence interval:
 -0.9979274  0.9373945
sample estimates:
       cor 
-0.6961061 


[1] "Test lewostronny"



	Pearson's product-moment correlation

data:  x and y
t = -1.3712, df = 2, p-value = 0.1519
alternative hypothesis: true correlation is less than 0
99 percent confidence interval:
 -1.0000000  0.8989349
sample estimates:
       cor 
-0.6961061 


[1] "Test prawostronny"



	Pearson's product-moment correlation

data:  x and y
t = -1.3712, df = 2, p-value = 0.8481
alternative hypothesis: true correlation is greater than 0
99 percent confidence interval:
 -0.9965887  1.0000000
sample estimates:
       cor 
-0.6961061 


# Testy dla dwóch populacji

### Jaki test wybrac
![](images/jaki_test_wybrac.PNG)

## Test sparowane
### Test Z sparowane

In [24]:
sigma <- 4
x1 <- c(1, 2, 3, 4, 5, 6) #populacja 1
x2 <- c(3, 3, 2, 1, 2, 3) #populacja 2
x <- x1 - x2
n <- length(x)

if(n <= 30) {
    print("Założenia CTL nie są spełnione! (n <= 30)")
} 

#H0: mu = 0
poziom_istotnosci <- 0.01
z_test <- (mean(x) - 0) / (sigma / sqrt(n))
#z_test <- 3
print(paste("Wartość statystyki testowej:", z_test))

#H1: mu < 0
cat("H1: mu < 0\n")
p_val_mniejsze <- pnorm(q = z_test, lower.tail = T)
cat(sprintf("p-wartość lewy ogon: %f
Zbiór krytyczny: (-Inf, %f)\n\n", p_val_mniejsze, qnorm(p=poziom_istotnosci)))


#H1: mu > 0
p_val_wieksze <- pnorm(q = z_test, lower.tail = F)
cat("H1: mu > 0\n")
cat(sprintf("p-wartość prawy ogon: %f
Zbiór krytyczny: (%f, Inf)\n\n", p_val_wieksze, qnorm(p=1-poziom_istotnosci)))

#H1: mu != 0
p_val_rozne <- 2 * pnorm(q=abs(z_test), lower.tail = F)
cat("H1: mu != 0\n")
cat(sprintf("p-wartość oba ogony: %f
Zbiór krytyczny: (-Inf, %f) v (%f, Inf)\n\n", p_val_rozne, qnorm(p=poziom_istotnosci/2), qnorm(p=1-poziom_istotnosci/2)))

[1] "Zalozenia CTL nie sa spelnione!"
[1] "Wartosc statystyki testowej: 0.71443450831176"
H1: mu < 0
p-wartosc lewy ogon: 0.762521
Zbiór krytyczny: (-Inf, -2.326348)

H1: mu > 0
p-wartosc prawy ogon: 0.237479
Zbiór krytyczny: (2.326348, Inf)

H1: mu != 0
p-wartosc oba ogony: 0.474959
Zbiór krytyczny: (-Inf, -2.575829) v (2.575829, Inf)



## Test Z niesparowane
![](images/test_z_sparowane.PNG)

In [18]:
mean1 <- 1
mean2 <- 2
sigma1 <- 1
sigma2 <- 2
n1 <- 50
n2 <- 50
poziom_istotnosci <- 0.01
z_test <- (mean1 - mean2) / sqrt(sigma1^2/n1 + sigma2^2/n2)

if(n1 <= 30 | n2 <= 30) {
    print("Założenia CTL nie są spełnione!")
}

print("H0: mean1 = mean2")
print(paste("Z_{mean1 - mean2} =", z_test))

#H0: mean1 = mean2

#H1: mean1 < mean2
p_val_lewo <- pnorm(q=z_test, lower.tail = T)
cat(sprintf("
H1: mean1 < mean2
p-wartosc: %f
Zbior krytyczny: (-Inf, %f)
", p_val_lewo, qnorm(poziom_istotnosci)))

#H1: mean1 > mean2
p_val_prawo <- pnorm(q=z_test, lower.tail = F)
cat(sprintf("
H1: mean1 > mean2
p-wartosc: %f
Zbior krytyczny: (-Inf, %f)
", p_val_prawo, qnorm(1-poziom_istotnosci)))

#H1: mean1 != mean2
p_val_obu <- 2 * pnorm(q=abs(z_test), lower.tail = F)
cat(sprintf("
H1: mean1 = mean2
p-wartosc: %f
Zbior krytyczny: (-Inf, %f) v (%f, Inf)
", p_val_obu, qnorm(poziom_istotnosci/2), qnorm(1-poziom_istotnosci/2)))

[1] "H0: mean1 = mean2"
[1] "Z_{mean1 - mean2} = -3.16227766016838"

H1: mean1 < mean2
p-wartosc: 0.000783
Zbior krytyczny: (-Inf, -2.326348)

H1: mean1 > mean2
p-wartosc: 0.999217
Zbior krytyczny: (-Inf, 2.326348)

H1: mean1 = mean2
p-wartosc: 0.001565
Zbior krytyczny: (-Inf, -2.575829) v (2.575829, Inf)


## Test T
### Dla równych wariancji
![](images/test_t_rowne_wariancje.PNG)

### Test dla podanych statystyk

In [19]:
x1_sr <- 1
s1 <- 1
n1 <- 31 

x2_sr <- 2
s2 <- 2
n2 <- 31

if(n1 <= 30 | n2 <= 30){
    print("Założenia CLT nie są spełnione! (n1 > 30 i n2 > 30)")
}

df <- n1 + n2 - 2

stat_test <- (x1_sr - x2_sr) / sqrt(
    ((n1 - 1) * s1^2 + (n2 - 1) * s2^2) / (n1 + n2 - 2) * (1 / n1 + 1 / n2)
)
print(paste("T_(n1 + n2 - 2) =", stat_test))

poziom_istotnosci <- 0.01

cat("\nTest lewostronny\n")
cat(paste("p-wartosc", pt(stat_test, df=df), '\n'))
obsz_kryt_lewo <- qt(poziom_istotnosci, df=df)
cat(sprintf("Zbior krytyczny: (-Inf, %f)\n", obsz_kryt_lewo))

cat("\nTest prawostronny\n")
cat(paste("p-wartosc", 1 - pt(stat_test, df=df), '\n'))
obsz_kryt_prawo <- qt(1 - poziom_istotnosci, df=df)
cat(sprintf("Zbior krytyczny: (%f, Inf)\n", obsz_kryt_prawo))

cat("\nTest obustronny\n")
cat(paste("p-wartosc", pt(stat_test, df=df) + 1 - pt(-stat_test, df=df), '\n'))
obsz_kryt_obu <- qt(poziom_istotnosci / 2, df=df)
cat(sprintf("Zbior krytyczny: (-Inf, %f) v (%f, Inf)\n", obsz_kryt_obu, -obsz_kryt_obu))

x <- wygeneruj_dane(1,1,5)
y <- wygeneruj_dane(2,2,10)
test <- t.test(x, y, var.equal = T, conf.level = 0.99)
print(test)
test$statistic

[1] "T_(n1 + n2 - 2) = -2.48997991959775"

Test lewostronny
p-wartosc 0.00778146781419984 
Zbior krytyczny: (-Inf, -2.390119)

Test prawostronny
p-wartosc 0.9922185321858 
Zbior krytyczny: (2.390119, Inf)

Test obustronny
p-wartosc 0.0155629356283997 
Zbior krytyczny: (-Inf, -2.660283) v (2.660283, Inf)

	Two Sample t-test

data:  x and y
t = -1.0408, df = 13, p-value = 0.3169
alternative hypothesis: true difference in means is not equal to 0
99 percent confidence interval:
 -3.894101  1.894101
sample estimates:
mean of x mean of y 
        1         2 



t 
-1.040833

## TEST DLA PODANYCH DANYCH
### Dla rownych wariancji

In [20]:
rowne_wariancje <- T

### Dla roznych wariancji

In [21]:
rowne_wariancje <- F

In [22]:
x <- wygeneruj_dane(1, 1, 31)
y <- wygeneruj_dane(2, 2, 31)

poziom_istotnosci <- 0.01
sparowany <- T #CZY SPAROWANY


#sprawdzenie zalozen CLT
if (length(x) > 30 & length(y) > 30){
    print("Zalozenia CTL spelnione (n1 > 30 i n2 > 30)")
}else {
    print("Zalozenia CTL nie sa spelnione! n1 <= 30 lub n2 <= 30")
}

test <- function(x, y, alternative) {
    res <- t.test(x, y, 
              paired = sparowany, 
              conf.level = 1 - poziom_istotnosci, 
              alternative = alternative,
             var.equal = rowne_wariancje
             )
    print(res)
    cat(paste("Wartosc statystyki testowej:", res$statistic, '\n'))
    if(res$p.value < poziom_istotnosci){
        cat("p-wartosc mniejsza od poziomu istotnosci. Odrzucamy H0")
    }else{
        cat("p-wartosc wieksza od poziomu istotnosci. Brak podstaw do odrzucenia H0")
    }
}

cat("\n\n----------------\nTest obustronny\n")
test(x, y, "two.sided")

cat("\n\n----------------\nTest lewostronny\n")
test(x, y, "less")

cat("\n\n----------------\nTest prawostronny\n")
test(x, y, "greater")

[1] "Zalozenia CTL spelnione (n1 > 30 i n2 > 30)"


----------------
Test obustronny

	Paired t-test

data:  x and y
t = -5.5678, df = 30, p-value = 4.691e-06
alternative hypothesis: true difference in means is not equal to 0
99 percent confidence interval:
 -1.4939138 -0.5060862
sample estimates:
mean of the differences 
                     -1 

Wartosc statystyki testowej: -5.56776436283002 
p-wartosc mniejsza od poziomu istotnosci. Odrzucamy H0

----------------
Test lewostronny

	Paired t-test

data:  x and y
t = -5.5678, df = 30, p-value = 2.346e-06
alternative hypothesis: true difference in means is less than 0
99 percent confidence interval:
       -Inf -0.5586628
sample estimates:
mean of the differences 
                     -1 

Wartosc statystyki testowej: -5.56776436283002 
p-wartosc mniejsza od poziomu istotnosci. Odrzucamy H0

----------------
Test prawostronny

	Paired t-test

data:  x and y
t = -5.5678, df = 30, p-value = 1
alternative hypothesis: true difference in me

# Regresja liniowa

## Założenia
![](images/zal_reg_lin.PNG)
### Twierdzenia
![](images/reg_tw1.PNG)
![](images/reg_tw2.PNG)
![](images/reg_tw3.PNG)
### Reszta
![](images/reszta.PNG)
### SSE
![](images/SSE.PNG)
### SSR
![](images/SSR.PNG)
### SST
![](images/SST.PNG)
### RMS
![](images/RMS.PNG)
![](images/RMS_NIEOBCIAZONE.PNG)
### Współczynnik determinacji
![](images/wsp_det.PNG)